In [ ]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
import sys

sys.path.append('../scripts')
sys.path.append('../')

from scripts import graph_osm_loader
from scripts import pipeline, centroids_graph_builder, clustering
from scripts.path_findings import excraction_pfa, dijkstra_pfa
from scripts import utils
import scripts

In [ ]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

In [ ]:
points = utils.read_points(GRAPH_ID, g, num=1000)

In [ ]:
cms_resolver = clustering.LouvainCommunityResolver(resolution=25)
cg = centroids_graph_builder.CentroidGraphBuilder().build(g, cms_resolver)

baseline = dijkstra_pfa.Dijkstra(g)

test_alg = excraction_pfa.ExtractionPfa(
    g = g,
    cg=cg,
    upper=dijkstra_pfa.Dijkstra(cg.g),
    down=dijkstra_pfa.Dijkstra(g)
)

In [ ]:
stat = pipeline.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=10
).compare()

In [ ]:
stat